In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
import sys
sys.path.insert(0, "/workspace/ojitong")

# Login to naver

In [76]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

from pyvirtualdisplay import Display
import Xlib.display
# 가상 디스플레이 시작
display = Display(visible=0, size=(1920, 1080), backend="xvfb", use_xauth=True)
display.start()
os.environ["DISPLAY"] = display.new_display_var

import pyperclip
import pyautogui

pyperclip.set_clipboard("xclip")
pyautogui._pyautogui_x11._display = Xlib.display.Display(os.environ['DISPLAY'])

def login_to_naver(username, password):
    # DBus 관련 환경 변수를 설정하지 않음
    os.environ["DBUS_SESSION_BUS_ADDRESS"] = "/dev/null"
    os.environ["NO_AT_BRIDGE"] = "1"
    
    # 크롬 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--headless=new')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--remote-debugging-port=9222')
    chrome_options.add_argument('--window-size=1920,1080') # https://stackoverflow.com/a/60505472
    chrome_options.add_argument("start-maximized")
    chrome_driver_path = '/usr/local/bin/chromedriver'

    # 크롬 드라이버 초기화
    chrome_service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    driver.get('https://nid.naver.com/nidlogin.login')

    # 로딩 대기
    time.sleep(2)

    # 아이디 입력 필드 찾기
    driver.execute_script(
        f"document.querySelector('input[id=\"id\"]').setAttribute('value', '{username}')"
    )
    time.sleep(1)

    # 패스워드 입력 필드 찾기
    driver.execute_script(
        f"document.querySelector('input[id=\"pw\"]').setAttribute('value', '{password}')"
    )
    time.sleep(1)
    
    # 로그인 버튼 클릭
    login_btn = driver.find_element(By.ID, "log.login")
    login_btn.click()

    # 로그인 후의 추가적인 처리가 필요한 경우를 위해 대기
    time.sleep(5)

    return driver

In [77]:
username = "tech_subworkers"
password = "Workersubway1!"
nickname = "subway__onda"

In [78]:
driver = login_to_naver(username, password)

In [85]:
from typing import Any, Dict
from selenium.webdriver.common.action_chains import ActionChains

def publish_blog_post(driver: Any, nickname: str, contents: Dict[str, Any]) -> None :
    
    actions = ActionChains(driver)
    
    # 글쓰기 페이지로 이동
    driver.get(f'https://blog.naver.com/{nickname}?Redirect=Write&')

    time.sleep(2)

    # 새 창으로 이동 (블로그 글쓰기 창이 새 탭/창에서 열렸다고 가정)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)

    # iframe으로 포커스 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'mainFrame')))
    time.sleep(2)

     # "작성 중인 글이 있습니다." 팝업 처리
    try:
        cancel_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div[3]/div/div/div[1]/div/div[4]/div[2]/div[3]/button[1]')))
        cancel_button.click()
    except Exception as e:
        print("* No draft popup appeared.")

    # 우측 팝업 처리
    try:
        close_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div[3]/div/div/div[1]/div/div[1]/article/div/header/button')))
        close_button.click()
    except Exception as e:
        print("* No right side popup appeared.")

    time.sleep(2)

    # 글쓰기 페이지 내 요소들의 XPATH

    title_field_xpath = '/html/body/div[1]/div/div[3]/div/div/div[1]/div/div[1]/div[2]/section/article/div[1]/div[1]/div/div/p/span[2]'
    content_field_xpath = '/html/body/div[1]/div/div[3]/div/div/div[1]/div/div[1]/div[2]/section/article/div[2]/div/div/div/div/p'
    save_button_xpath = '/html/body/div[1]/div/div[1]/div/div[2]/button[1]'
    text_image_xpath = '/html/body/div[1]/div/div[3]/div/div/div[1]/div/header/div[1]/ul/li[17]/button'
    image_keyword_xpath = '/html/body/div[1]/div/div[3]/div/div/div[1]/div/div[1]/aside/div/div[1]/input'
    first_image_xpath = '/html/body/div[1]/div/div[3]/div/div/div[1]/div/div[1]/aside/div/div[3]/div/ul/div/li[1]/div/div[2]'


    # # 이미지 넣기(글감 무료 이미지 활용)
    # #  글감 버튼 클릭
    # image_button = driver.find_element(By.XPATH, text_image_xpath)
    # image_button.click()
    # time.sleep(1)

    # # 키워드 입력 후 이미지 검색
    # keyword_field = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, image_keyword_xpath)))
    # pyperclip.copy(contents['Title'])
    # time.sleep(1)
    # keyword_field.click()
    # pyautogui.hotkey('ctrl', 'v')  # 클립보드 내용 붙여넣기
    # pyautogui.press('enter')
    # time.sleep(2)

    # #  첫 이미지 클릭
    # login_button = driver.find_element(By.XPATH, first_image_xpath)
    # login_button.click()


    # 제목 입력
    title_field = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, title_field_xpath)))
    title_field.click()
    actions.send_keys(contents['Title'])
    actions.perform()
    time.sleep(1)
    print("* Title is filled")

    # 내용 입력
    content_field = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, content_field_xpath)))
    content_field.click()
    actions.send_keys(contents['Contents'])
    actions.perform()
    print("* Content is filled")

    # "저장" 버튼 클릭
    save_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, save_button_xpath)))
    save_button.click()
    print("* Savebutton is clicked")

    # 저장 후의 처리가 필요한 경우를 위해 대기
    time.sleep(2)

    # 글쓰기 작업 종료 이후 드라이버 종료
    # driver.quit()  # 드라이버 종료

In [86]:
contents = {"Title" : "테스트제목", "Contents" : "테스트내용", "Hashtags": "#해시태그"}

In [87]:
publish_blog_post(driver, nickname, contents)

* No right side popup appeared.
* Title is filled
* Content is filled
